In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('../Data/dataset.csv')

dataset.head(3)

,B0043GG,B0043HH,B0043II,Protein,Leukocyte,SG,Bilirubin,Acidity,Nitrite,Urobilinogen,...,Urin_color_Dk.Yellow,Urin_color_Lt. Yellow,Urin_color_Lt.Orange,Urin_color_Orange,Urin_color_Other,Urin_color_Red,Urin_color_Reddish,Urin_color_Straw,Urin_color_Yellow,Urin_color_yellow
0,9.3,251.5,4.3,1,119,1.020,0.1,5.0,103842,0.7,...,0,0,0,0,0,0,0,0,1,0
1,5.1,548.3,9.1,4,106,1.020,0.0,5.0,108098,0.7,...,0,0,0,0,0,0,0,0,1,0
2,12.2,1590.0,23.5,6,657,1.016,0.1,5.0,27515,0.7,...,0,0,0,0,0,0,0,0,1,0


In [3]:
dataset.corr()['Glucose'].abs().sort_values(ascending=False)

Glucose                  1.000000
SG                       0.317661
Ketone_body              0.286262
Bilirubin                0.240392
Leukocyte                0.199949
Protein                  0.145639
Urin_color_Yellow        0.125012
B0043HH                  0.120803
Acidity                  0.100742
Urin_color_Orange        0.078550
Occult_blood             0.076694
Urobilinogen             0.076340
Urin_color_Red           0.062400
Urin_color_Dk.Yellow     0.053695
Urin_color_yellow        0.044257
B0043II                  0.041301
Urin_color_Reddish       0.025791
B0043GG                  0.024758
Urin_color_Lt. Yellow    0.015962
Urin_color_DK.Orange     0.015574
Urin_color_Lt.Orange     0.014906
Urin_color_Brown         0.009365
Urin_color_Amber         0.007816
Urin_color_Dk.Orange     0.006484
Urin_color_DK.Yellow     0.005147
Urin_color_Dk.Red        0.004626
Urin_color_Other         0.004221
Nitrite                  0.004171
Urin_color_Straw         0.003387
Urin_color_Dk.

In [5]:
# Selecing columns

dataset = dataset[['SG', 'Ketone_body', 'Bilirubin', 'Leukocyte',
                   'Protein', 'Urin_color_Yellow', 'B0043HH',
                   'Acidity', 'Glucose']]

dataset.head(3)

,SG,Ketone_body,Bilirubin,Leukocyte,Protein,Urin_color_Yellow,B0043HH,Acidity,Glucose
0,1.020,1,0.1,119,1,1,251.5,5.0,16
1,1.020,1,0.0,106,4,1,548.3,5.0,65
2,1.016,3,0.1,657,6,1,1590.0,5.0,79


In [6]:
dataset.shape

(74337, 9)

In [7]:
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

X.shape, y.shape

((74337, 8), (74337,))

# Train, Test Split

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2022)

X_train.shape, X_test.shape

((59469, 8), (14868, 8))

In [13]:
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2022)

# X_train.shape, X_val.shape

((47575, 3), (11894, 3))

# Data Scaling

In [11]:
from sklearn.preprocessing import StandardScaler

In [12]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
# X_val = ss.transform(X_val)
X_test = ss.transform(X_test)

In [12]:
X_train[:5, :]

array([[-0.59688789, -0.6161976 ,  0.19038558,  0.27419382, -0.4568329 ,
        -0.53870365, -0.68592602],
       [-0.34512929, -0.55352856, -0.54462091, -0.87977981, -0.44443037,
        -0.48245593, -0.55110695],
       [-1.60392227, -0.65403552,  0.55788882, -0.87977981, -0.48411847,
        -0.55628106,  1.72284137],
       [ 0.91366369, -0.55352856, -0.54462091, -0.87977981, -0.44443037,
        -0.48245593,  2.00596142],
       [-0.84864648, -0.60792056,  0.55788882, -0.49512194,  0.03926827,
        -0.40863081, -0.12867386]])

# Modeling

## 1. RandomForestRegressor

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold

In [14]:
params = {
    'n_estimators' : range(100, 200, 5),
    'verbose' : [True]
}

In [15]:
rfr = RandomForestRegressor(n_jobs=8, random_state=2022)

In [17]:
# 0값 확인
# 0값 존재 -> MAPE 사용 불가 (실제 y값으로 나누기 때문(y > 0 필수))

len(dataset.loc[dataset.Glucose == 0])

246

In [18]:
kfold = KFold(5, shuffle=True, random_state=2022)

### Learning

In [19]:
grid = GridSearchCV(rfr, params, n_jobs=8, cv=kfold, refit=True)
grid.fit(X_train, y_train)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 195 out of 195 | elapsed:    3.5s finished


GridSearchCV(cv=KFold(n_splits=5, random_state=2022, shuffle=True),
             estimator=RandomForestRegressor(n_jobs=8, random_state=2022),
             n_jobs=8,
             param_grid={'n_estimators': range(100, 200, 5), 'verbose': [True]})

In [20]:
grid.best_params_

{'n_estimators': 195, 'verbose': True}

In [21]:
y_pred = grid.best_estimator_.predict(X_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 195 out of 195 | elapsed:    0.0s finished


### Evaluating

In [22]:
from sklearn.metrics import mean_absolute_percentage_error, \
                            mean_absolute_error, mean_squared_error

In [23]:
mean_absolute_percentage_error(y_test, y_pred)

2997435196544829.0

In [24]:
mean_absolute_error(y_test, y_pred)

509.57522175635756

In [25]:
np.sqrt(mean_squared_error(y_test, y_pred))

700.2317971070996

### Comparising y_pred, y_result

In [26]:
comparison = pd.DataFrame({'prediction' : y_pred, 'result' : y_test.values.ravel()})

In [27]:
comparison

,prediction,result
0,1143.088211,92
1,119.713546,303
2,46.000000,46
3,1046.284745,2125
4,94.989744,95
...,...,...
14863,156.419191,303
14864,1164.224046,92
14865,74.051282,71
14866,79.000000,79


### Saving Model

In [28]:
import pickle

In [29]:
saved_model = pickle.dump(grid.best_estimator_, open('../Model/{0}_RFT_n_{1}.pkl' \
    .format(dataset.columns[-1], grid.best_params_.get('n_estimators')), 'wb'),
                          protocol=pickle.HIGHEST_PROTOCOL)

### Loading Model

In [30]:
loaded_model = pickle.load(open('../Model/{0}_RFT_n_{1}.pkl' \
    .format(dataset.columns[-1], grid.best_params_.get('n_estimators')), 'rb'))

In [31]:
y_pred = loaded_model.predict(X_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 195 out of 195 | elapsed:    0.0s finished


In [32]:
mean_absolute_error(y_test, y_pred)

509.57522175635756

In [33]:
np.sqrt(mean_squared_error(y_test, y_pred))

700.2317971070996